<a href="https://colab.research.google.com/github/JorgePere27/Music_pref_project/blob/main/Music_pref_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Music Preferences Project

# Contents <a id='back'></a>

* [Introducción](#intro)
* [Etapa 1. Descripción de los datos](#data_review)
    * [Conclusions](#data_review_conclusions)
* [Etapa 2. Data preprocessing](#data_preprocessing)
    * [2.1 Estilo del encabezado](#header_style)
    * [2.2 Valores ausentes](#missing_values)
    * [2.3 Duplicados](#duplicates)
    * [2.4 Conclusiones](#data_preprocessing_conclusions)
* [Etapa 3. Prueba de hipótesis](#hypotheses)
    * [3.1 Hipótesis 1: comparar el comportamiento del usuario en las dos ciudades](#activity)
    * [3.2 Hipótesis 2: música al principio y al final de la semana](#week)
    * [3.3 Hipótesis 3: preferencias de género en Springfield y Shelbyville](#genre)
* [Conclusiones](#end)



## Introducción <a id='intro'></a>
Siempre que investiguemos, necesitamos formular hipótesis que después podamos probar. A veces aceptamos estas hipótesis; otras, las rechazamos. Para tomar las decisiones correctas, una empresa debe ser capaz de entender si está haciendo las suposiciones correctas.

En este proyecto, se compararán las preferencias musicales de las ciudades de Springfield y Shelbyville. Estudiando datos recabados para probar las hipótesis siguientes y comparar el comportamiento del usuario de esas dos ciudades.

### Objetivo:
Probar las siguientes hipótesis:
1. La actividad de los usuarios difiere según el día de la semana y dependiendo de la ciudad.
2. Los lunes por la mañana, los habitantes de Springfield y Shelbyville escuchan diferentes géneros. Lo mismo ocurre con los viernes por la noche.
3. Los oyentes de Springfield y Shelbyville tienen preferencias distintas. En Springfield prefieren el pop mientras que en Shelbyville hay más aficionados al rap.

### Etapas
Los datos del comportamiento del usuario se almacenan en el archivo `music_project_en.csv`. No existe información sobre la calidad de los datos así que sera necesario examinarlos antes de poder probar las hipótesis.

Primero, se evaluará la calidad de los datos y para saber si existen problemas que sean significativos.

El proyecto consistirá de tres etapas:
 1. Descripción de los datos
 2. Preprocesamiento de datos
 3. Prueba de hipótesis

[Volver a Contenidos](#back)

In [1]:
import pandas as pd
from google.colab import drive

drive.mount("/content/Drive")

Mounted at /content/Drive


In [2]:
pd.set_option('display.max_columns',None)
global df_banco, resultados

In [3]:
df_music = pd.read_csv('/content/Drive/MyDrive/Tripleten projects/music_project_en.csv')
df_music.head()

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday


In [4]:
df_music.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63736 non-null  object
 2   artist    57512 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


La tabla contiene siete columnas. Todas almacenan el mismo tipo de datos: objeto.

De acuerdo con la documentación:
- `'userID'` — identificador del usuario
- `'Track'` — título de la pista
- `'artist'` — nombre del artista
- `'genre'` — género
- `'City'` — ciudad del usuario
- `'time'` — el periodo de tiempo exacto en que se reprodujo la pista
- `'Day'` — día de la semana

Podemos ver tres problemas con el estilo en los nombres de las columnas:
1. Algunos nombres están en mayúsculas, otros en minúsculas.
2. Hay algunos espacios en algunos nombres.
3. La columna time requiere mas informacion, sobre especificamente que informacion contien y en que unidad esta.

El número de valores de las columnas es diferente. Esto significa que los datos contienen valores ausentes.

### Conclusiones <a id='data_review_conclusions'></a>

Cada fila de la tabla almacena datos de la pista que fue reproducida. Algunas columnas describen la pista en sí: su título, el artista y el género. El resto transmite la información del usuario: la ciudad de la que viene, el tiempo que ha reproducido la pista.

Está claro que los datos son suficientes para probar la hipótesis. Sin embargo, hay valores ausentes.

Para continuar, necesitamos preprocesar los datos.

## Etapa 2. Preprocesamiento de datos <a id='data_preprocessing'></a>
Corregimos el formato en los encabezados de las columnas y nos ocupamos de los valores ausentes. Después, comprobaremos si hay duplicados en los datos.

In [6]:
#Verificamos los nombres de las columnas del dataframe
df_music.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Cambiamos los nombres de las columnas de acuerdo con las reglas del buen estilo:
* Si el nombre tiene varias palabras, utiliza snake_case
* Todos los caracteres deben ser minúsculas
* Elimina los espacios

In [7]:
# renombramos las columnas
df_music= df_music.rename(
    columns={'  userID':'user_ID',
             'Track':'track',
             'artist':'artist',
             'genre':'genre',
             '  City  ':'city',
             'time':'playtime',
             'Day':'day',})

#comprobamos los cambios realizados
df_music.columns

Index(['user_ID', 'track', 'artist', 'genre', 'city', 'playtime', 'day'], dtype='object')

### Valores ausentes <a id='missing_values'></a>
Primero encontramos el número de valores ausentes en la tabla. Para ello, utilizamos dos métodos pandas:

In [9]:
# calculando valores ausentes
print(df_music.isna().sum())

user_ID        0
track       1343
artist      7567
genre       1198
city           0
playtime       0
day            0
dtype: int64


No todos los valores ausentes afectan a la investigación. Por ejemplo, los valores ausentes en la pista y artista no son cruciales. Simplemente puedes reemplazarlos por marcadores claros.

Pero los valores ausentes en 'genre' pueden afectar la comparación entre las preferencias musicales de Springfield y Shelbyville. En la vida real, sería útil saber las razones por las cuales hay datos ausentes e intentar recuperarlos. Pero no tenemos esa oportunidad en este proyecto. Así que tendrás que:
* Rellenar esos valores ausentes con marcadores
* Evaluar cuánto podrían afectar los valores ausentes a tus cómputos.


Reemplazar los valores ausentes en 'track', 'artist', y 'genre' con la string 'unknown'. Para ello, crea la lista columns_to_replace, recórrela con un bucle for y reemplaza los valores ausentes en cada una de las columnas:

In [11]:
# recorriendo los nombres de las columnas y reemplazando los valores ausentes con 'unknown'
columns_to_replace=['track','artist','genre']
for column in columns_to_replace:
    df_music[column]=df_music[column].fillna('unknown')

In [13]:
# Comprobamos de nuevo contando valores ausentes
print(df_music.isna().sum())

user_ID     0
track       0
artist      0
genre       0
city        0
playtime    0
day         0
dtype: int64


### Duplicados <a id='duplicates'></a>
Encuentra el número de duplicados obvios en la tabla utilizando un comando:

In [14]:
# contando duplicado obvios
print(df_music.duplicated().sum())

3826


In [16]:
# eliminando duplicados obvios
df_music=df_music.drop_duplicates().reset_index(drop=True)

In [17]:
#verificamos los cambios realizados
print(df_music.duplicated().sum())

0


Ahora deshazte de los duplicados implícitos en la columna genre. Por ejemplo, el nombre de un género se puede escribir de varias formas. Dichos errores también pueden afectar a resultado.

Imprime una lista de nombres únicos de géneros, ordenados en orden alfabético. Cómo se hace:
* Recupera la deseada columna DataFrame
* Aplícale un método de orden
* Para la columna ordenada, llama al método que te devolverá todos los valores de columna únicos

In [18]:
# inspeccionando los nombres de géneros únicos
df_music.sort_values(by=['genre'])['genre'].unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'ambient', 'americana', 'animated', 'anime',
       'arabesk', 'arabic', 'arena', 'argentinetango', 'art', 'audiobook',
       'avantgarde', 'axé', 'baile', 'balkan', 'beats', 'bigroom',
       'black', 'bluegrass', 'blues', 'bollywood', 'bossa', 'brazilian',
       'breakbeat', 'breaks', 'broadway', 'cantautori', 'cantopop',
       'canzone', 'caribbean', 'caucasian', 'celtic', 'chamber',
       'children', 'chill', 'chinese', 'choral', 'christian', 'christmas',
       'classical', 'classicmetal', 'club', 'colombian', 'comedy',
       'conjazz', 'contemporary', 'country', 'cuban', 'dance',
       'dancehall', 'dancepop', 'dark', 'death', 'deep', 'deutschrock',
       'deutschspr', 'dirty', 'disco', 'dnb', 'documentary', 'downbeat',
       'downtempo', 'drum', 'dub', 'dubstep', 'eastern', 'easy',
       'electronic', 'electropop', 'emo', 'entehno', 'epicmetal',
       'estrada', 'ethnic', 'eurofo

Busca en la lista para encontrar duplicados implícitos del género hiphop. Estos pueden ser nombres escritos incorrectamente o nombres alternativos para el mismo género.

Verás los siguientes duplicados implícitos:
* `hip`
* `hop`
* `hip-hop`

Para deshacerte de ellos, declara la función replace_wrong_genres() con dos parámetros:
* `wrong_genres=` — la lista de duplicados
* `correct_genre=` — la string con el valor correcto

La función debería corregir los nombres en la columna 'genre' de la tabla df, es decir, remplaza cada valor de la lista wrong_genres con el valor en correct_genre.

In [21]:
# función para reemplazar duplicados implícitos
def replace_wrong_genres (wrong_genres,correct_genre):
    for wrong_genre in wrong_genres:
        df_music['genre']=df_music['genre'].replace(wrong_genre,correct_genre)

Llama a replace_wrong_genres() y pásale argumentos para que retire los duplicados implícitos (hip, hop y hip-hop) y los reemplace por hiphop:

In [22]:
# eliminando duplicados implícitos
replace_wrong_genres(['hip','hop','hip-hop'],'hiphop')

In [24]:
# comprobamos los cambios realizados
df_music.sort_values(by=['genre'])['genre'].unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'ambient', 'americana', 'animated', 'anime',
       'arabesk', 'arabic', 'arena', 'argentinetango', 'art', 'audiobook',
       'avantgarde', 'axé', 'baile', 'balkan', 'beats', 'bigroom',
       'black', 'bluegrass', 'blues', 'bollywood', 'bossa', 'brazilian',
       'breakbeat', 'breaks', 'broadway', 'cantautori', 'cantopop',
       'canzone', 'caribbean', 'caucasian', 'celtic', 'chamber',
       'children', 'chill', 'chinese', 'choral', 'christian', 'christmas',
       'classical', 'classicmetal', 'club', 'colombian', 'comedy',
       'conjazz', 'contemporary', 'country', 'cuban', 'dance',
       'dancehall', 'dancepop', 'dark', 'death', 'deep', 'deutschrock',
       'deutschspr', 'dirty', 'disco', 'dnb', 'documentary', 'downbeat',
       'downtempo', 'drum', 'dub', 'dubstep', 'eastern', 'easy',
       'electronic', 'electropop', 'emo', 'entehno', 'epicmetal',
       'estrada', 'ethnic', 'eurofo

### Conclusiones <a id='data_preprocessing_conclusions'></a>
Detectamos tres problemas con los datos:

- Estilos de encabezados incorrectos
- Valores ausentes
- Duplicados obvios e implícitos

Los encabezados han sido eliminados para conseguir que el procesamiento de la tabla sea más sencillo.

Todos los valores ausentes han sido reemplazados por 'unknown'. Pero todavía tenemos que ver si los valores ausentes en 'genre' afectan a nuestros cálculos.

La ausencia de duplicados hará que los resultados sean mas precisos y fáciles de entender.

Ahora ya podemos continuar probando las hipótesis.

## Etapa 3. Prueba de hipótesis <a id='hypotheses'></a>

### Hipótesis 1: comparar el comportamiento del usuario en las dos ciudades <a id='activity'></a>

De acuerdo con la primera hipótesis, los usuarios de Springfield y Shelbyville escuchan música de forma distinta. Comprueba esto utilizando los datos de tres días de la semana: lunes, miércoles y viernes.

* Divide a los usuarios en grupos por ciudad.
* Compara cuántas pistas reprodujo cada grupo el lunes, el miércoles y el viernes.


Evalúa la actividad del usuario en cada ciudad. Agrupa los datos por ciudad y encuentra el número de canciones reproducidas en cada grupo.


In [25]:
# contando las pistas reproducidas en cada ciudad
df_music.groupby('city').count()


,user_ID,track,artist,genre,playtime,day
city,,,,,,
Shelbyville,18512,18512,18512,18512,18512,18512
Springfield,42741,42741,42741,42741,42741,42741


Springfield ha reproducido más pistas que Shelbyville. Pero eso no implica que los ciudadanos de Springfield escuchen música más a menudo. Esta ciudad es simplemente más grande y hay más usuarios.

Ahora agrupa los datos por día de la semana y encuentra el número de pistas reproducidas el lunes, miércoles y viernes.

In [28]:
# calculando las pistas reproducidas en cada uno de los tres días
df_music.groupby('day').count()

,user_ID,track,artist,genre,city,playtime
day,,,,,,
Friday,21840,21840,21840,21840,21840,21840
Monday,21354,21354,21354,21354,21354,21354
Wednesday,18059,18059,18059,18059,18059,18059


El miércoles fue el día más silencioso de todos. Pero si consideramos las dos ciudades por separado podríamos llegar a una conclusión diferente.

Ya has visto cómo funciona el agrupar por ciudad o día. Ahora escribe la función que agrupará ambos.

Crea la función number_tracks() para calcular el número de canciones reproducidas en un determinado día y ciudad. Requerirá dos parámetros:
* día de la semana
* nombre de la ciudad

En la función, utiliza una variable para almacenar las filas de la tabla original, donde:
  * el valor de la columna 'day' es igual al parámetro de día
  * el valor de la columna 'city' es igual al parámetro de ciudad

Aplica un filtrado consecutivo con indexación lógica.

Después, calcula los valores de la columna 'user_id' en la tabla resultante. Almacena el resultado en la nueva variable. Recupera esta variable de la función.

In [29]:
# <creando la función number_tracks()>
# declararemos la función con dos parámetros: day=, city=.
# deja que la variable track_list almacene las filas df en las que
# el valor en la columna 'day' es igual al parámetro day= y, al mismo tiempo,
# el valor de la columna 'city' es igual al parámetro city= (aplica el filtrado consecutivo
# con indexación lógica).
# deja que la variable track_list_count almacene el número de valores de la columna 'user_id' en track_list
# (encontrado con el método count()).
# permite que la función devuelva un número: el valor de track_list_count.

# la función cuenta las pistas reproducidas en un cierto día y ciudad.
# primero recupera las filas del día deseado de la tabla,
# después filtra las filas de la ciudad deseada del resultado,
# entonces, encuentra el número de valores de 'user_id' en la tabla filtrada,
# y devuelve ese número.
# para ver lo que devuelve, envuelve la llamada de la función en print().

def number_tracks(day,city):
    track_list=df_music[df_music['day']==day]
    track_list=track_list[track_list['city']==city]
    track_list_count=track_list['user_ID'].count()
    print(track_list_count)

Llama a number_tracks() seis veces, cambiando los valores de los parámetros, para que recuperes los datos de ambas ciudades para cada uno de los tres días.

In [30]:
# el número de canciones reproducidas en Springfield el lunes
number_tracks('Monday','Springfield')

15740


In [31]:
# el número de canciones reproducidas en Shelbyville el lunes
number_tracks('Monday','Shelbyville')

5614


In [32]:
# el número de canciones reproducidas en Springfield el miércoles
number_tracks('Wednesday','Springfield')

11056


In [33]:
# el número de canciones reproducidas en Shelbyville el miércoles
number_tracks('Wednesday','Shelbyville')

7003


In [34]:
# el número de canciones reproducidas en Springfield el viernes
number_tracks('Friday','Springfield')

15945


In [35]:
# el número de canciones reproducidas en Shelbyville el viernes
number_tracks('Friday','Shelbyville')

5895


Utiliza pd.DataFrame para crear una tabla, donde
* Los nombres de las columnas son: `['city', 'monday', 'wednesday', 'friday']`
* Los datos son los resultados que conseguiste de number_tracks()`

In [36]:
# tabla con los resultados
datos={'city':['Springfield','Shelbyville'],
       'monday':[15740,5614],
       'wednesday':[11056,7003],
       'friday':[15945,5895]}
data=pd.DataFrame(datos)
data

,city,monday,wednesday,friday
0,Springfield,15740,11056,15945
1,Shelbyville,5614,7003,5895


**Conclusiones**

Los datos revelan las diferencias en el comportamiento de los usuarios:

- En Springfield, el número de canciones reproducidas alcanzan el punto máximo los lunes y viernes mientras que los miércoles hay un descenso de la actividad.
- En Shelbyville, al contario, los usuarios escuchan más música los miércoles. La actividad de los usuarios los lunes y viernes es menor.

Así que la primera hipótesis parece ser correcta.

### Hipótesis 2: música al principio y al final de la semana <a id='week'></a>

De acuerdo con la segunda hipótesis, los lunes por la mañana y los viernes por la noche los ciudadanos de Springfield escuchan géneros que difieren de aquellos que los usuarios de Shelbyville disfrutan.

Obtener tablas (asegúrate de que el nombre de tu tabla combinada encaja con el DataFrame dado en los dos bloques de código de abajo):
* Para Springfield — `spr_general`
* Para Shelbyville — `shel_general`

In [37]:
# obteniendo la tabla spr_general de las filas de df,
# donde los valores en la columna 'city' es 'Springfield'

spr_general = df_music[df_music['city'] == 'Springfield']

In [38]:
# obteniendo shel_general de las filas df,
# donde el valor de la columna 'city' es 'Shelbyville'

shel_general = df_music[df_music['city'] == 'Shelbyville']

Escribe la función genre_weekday() con cuatro parámetros:
* Una tabla para los datos
* El día de la semana
* La marca de fecha y hora en formato 'hh:mm'
* La marca de fecha y hora en formato 'hh:mm'

La función debería devolver información de los 15 géneros más populares de un día determinado en un período entre dos marcas de fecha y hora.

In [39]:
# declarando la función genre_weekday() con los parámetros day=, time1= y time2=. Debería
# devolver información sobre los géneros más populares de un determinado día a una determinada hora:

# 1) Deja que la variable genre_df almacene las filas que cumplen varias condiciones:
#    - el valor de la columna 'day' es igual al valor del argumento day=
#    - el valor de la columna 'time' es mayor que el valor del argumento time1=
#    - el valor en la columna 'time' es menor que el valor del argumento time2=
#    Utiliza un filtrado consecutivo con indexación lógica.

# 2) Agrupa genre_df por la columna 'genre', toma una de sus columnas,
#    y utiliza el método count() para encontrar el número de entradas por cada uno de
#    los géneros representados; almacena los Series resultantes en
#    la variable genre_df_count

# 3) Ordena genre_df_count en orden descendente de frecuencia y guarda el resultado
#    en la variable genre_df_sorted

# 4) Devuelve un objeto Series con los primeros 15 valores de genre_df_sorted - los 15
#    géneros más populares (en un determinado día, en un determinado periodo de tiempo)

def genre_weekday(df, day, time1, time2):
    # filtrado consecutivo
    # genre_df solo almacenará aquellas filas df en las que el día sea igual a day=
    genre_df = df[df['day'] == day]
    # genre_df solo almacenará aquellas filas df en las que el tiempo sea menos que time2=
    genre_df = genre_df[genre_df['playtime'] < time2]
    # genre_df solo almacenará aquellas filas df en las que el tiempo sea mayor que time1=
    genre_df = genre_df[genre_df['playtime'] > time1]
    # agrupa el DataFrame filtrado por la columna con los nombres de los géneros, toma la columna de género, y encuentra el número de filas por cada género con el método count()
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()
    # ordenaremos el resultado en orden descendente (por lo que los géneros más populares aparecerán primero en el objeto Series)
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False)
    # devolveremos el objeto Series que almacena los 15 géneros más populares en un día determinado en un periodo de tiempo determinado
    return genre_df_sorted[:15]

Compara los resultados de la función `genre_weekday()`para Springfield y Shelbyville el lunes por la mañana (de 7 a 11) y el viernes por la tarde (de 17:00 a 23:00):



In [40]:
# llamando a la función para el lunes por la mañana en Springfield (utilizando spr_general en vez de la tabla df)

genre_weekday(spr_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
classical      157
metal          120
jazz           100
folk            97
soundtrack      95
Name: genre, dtype: int64

In [41]:
# llamando a la función para el lunes por la mañana en Shelbyville (utilizando shel_general en vez de la tabla df)

genre_weekday(shel_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
world           36
rap             32
soundtrack      31
rnb             27
metal           27
Name: genre, dtype: int64

In [42]:
# llamando a la función para el viernes por la tarde en Springfield

genre_weekday(spr_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
classical      163
alternative    163
rusrap         142
jazz           111
unknown        110
soundtrack     105
rnb             90
metal           88
Name: genre, dtype: int64

In [43]:
# llamando a la función para el viernes por la tarde en Shelbyville

genre_weekday(shel_general, 'Friday', '17:00', '23:00')

genre
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
unknown         47
ruspop          47
soundtrack      40
metal           39
rap             36
Name: genre, dtype: int64

**Conclusión**

Habiendo comparado los 15 géneros más populares del lunes por la mañana podemos concluir lo siguiente:

1. Los usuarios de Springfield y Shelbyville escuchan música similar. Los cinco géneros más populares son los mismos, solo rock y electrónica han intercambiado posiciones.

2. En Springfield el número de valores ausentes resultaron ser tan altos que el valor 'unknown' llegó al décimo. Esto significa que los valores ausentes forman una parte considerable de los datos, lo que podría ser la base de la cuestión sobre la fiabilidad de nuestras conclusiones.

Para el viernes por la tarde, la situación es similar. Los géneros individuales varían algo pero, en general, los 15 más populares son parecidos en las dos ciudades.

De esta forma, la segunda hipótesis ha sido parcialmente demostrada:
* Los usuarios escuchan música similar al principio y al final de la semana.
* No hay una gran diferencia entre Springfield y Shelbyville. En ambas ciudades, el pop es el género más popular.

Sin embargo, el número de valores ausentes hace este resultado un tanto cuestionable. En Springfield, hay tantos que afectan a nuestros 15 más populares. De no faltarnos esos valores, las cosas podrían parecer diferentes.

### Hipótesis 3: preferencias de género en Springfield y Shelbyville <a id='genre'></a>

Hipótesis: Shelbyville ama la música rap. A los ciudadanos de Springfield les gusta más el pop.

Agrupa la tabla spr_general por género y encuentra el número de canciones reproducidas de cada género con el método count(). Después, ordena el resultado en orden descendente y guárdalo en spr_genres.

In [45]:
# agrupa la tabla spr_general por la columna 'genre',
# cuenta los valores 'genre' con count() en la agrupación,
# ordena el Series resultante en orden descendiente, y almacénalo en spr_genres
spr_genres = spr_general.groupby('genre')['genre'].count().sort_values(ascending=False)

In [48]:
# imprimiendo las 10 primeras filas de spr_genres
spr_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Ahora haz lo mismo con los datos de Shelbyville.

Agrupa la tabla shel_general por género y encuentra el número de canciones reproducidas de cada género. Después, ordena el resultado en orden descendente y guárdalo en la tabla shel_genres:

In [49]:
# agrupa la tabla shel_general por la columna 'genre',
# cuenta los valores 'genre' en el agrupamiento con count(),
# ordena el Series resultante en orden descendente y guárdalo en shel_genres
shel_genres = shel_general.groupby('genre')['genre'].count().sort_values(ascending=False)

In [50]:
# imprimiendo las 10 primeras filas de shel_genres
shel_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Conclusión**

La hipótesis ha sido parcialmente demostrada:
* La música pop es el género más popular en Springfield, tal como se esperaba.
* Sin embargo, la música pop ha resultado ser igual de popular en Springfield que en Shelbyville y el rap no estaba entre los 5 más populares en ninguna de las ciudades.

Hemos probado las siguientes tres hipótesis:

1. La actividad de los usuarios difiere dependiendo del día de la semana y de las distintas ciudades.
2. Los lunes por la mañana los residentes de Springfield y Shelbyville escuchan géneros distintos. Lo mismo ocurre con los viernes por la noche.
3. Los oyentes de Springfield y Shelbyville tienen distintas preferencias. En ambas ciudades, Springfield y Shelbyville, se prefiere el pop.

Tras analizar los datos, concluimos:

1. La actividad del usuario en Springfield y Shelbyville depende del día de la semana aunque las ciudades varían de diferentes formas.

La primera hipótesis ha sido aceptada completamente.

2. Las preferencias musicales no varían significativamente en el transcurso de la semana en Springfield y Shelbyville. Podemos observar pequeñas diferencias en el orden los lunes, pero:
* En Springfield y Shelbyville la gente lo que más escucha es la música pop.

Así que no podemos aceptar esta hipótesis. También debemos tener en cuenta que el resultado podría haber sido diferente si no fuera por los valores ausentes.

3. Resulta que las preferencias musicales de los usuarios de Springfield y Shelbyville son bastante parecidas.

La tercera hipótesis es rechazada. Si hay alguna diferencia en las preferencias no se puede observar en los datos.